# `sc_linac` Live

In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
from lcls_live.datamaps import get_datamaps
from lcls_live.archiver import lcls_archiver_restore

from lcls_live.tools import isotime

import matplotlib.pyplot as plt
import numpy as np

import os

In [3]:
# Pick a model and slice

MODEL = 'sc_diag0' # or sc_diag0 or sc_inj

BEGELE = 'BEGINNING'
#ENDELE = 'OTR0H04'
#ENDELE = 'ENDCOL0'
ENDELE = 'END'

# Bmad model

In [4]:
from pytao import Tao
import pandas as pd

In [5]:
tao = Tao(f'-init $LCLS_LATTICE/bmad/models/{MODEL}/tao.init  -slice  {BEGELE}:{ENDELE}')
tao.cmd('place floor bpm_orbit')
tao.cmd('place middle energy')
tao.cmd('sc floor -10 10')

def ele_info(ele):
    dat = tao.ele_head(ele)
    dat.update(tao.ele_gen_attribs(ele))
    return dat

def ele_table(match="*"):
    ix_ele = tao.lat_list(match, "ele.ix_ele", flags="-no_slaves")
    dat = list(map(ele_info, ix_ele))
    df = pd.DataFrame(dat, index=ix_ele)
    df.L.fillna(0, inplace=True)
    df['s_center'] = df['s'] - df['L']/2
    df['s_beginning'] = df['s'] - df['L']
    return  df

df = ele_table()

# Elements with device names
devices = df[df['alias'] != '']
#devices['name alias s'.split()]

## Datamaps, and all PVs needed

In [6]:
from copy import deepcopy
def filter_datamap(dm, bmad_names):
    bnames = dm.data['bmad_name'] 
    bmad_names = set(bmad_names)
    ix = bnames[[name in bmad_names for name in bnames]].index
    dm2 = deepcopy(dm)
    dm2.data = dm.data.loc[ix]
    return dm2

In [7]:
DM0 = get_datamaps(MODEL)
DM0.keys()

dict_keys(['bpms', 'cavities', 'correctors', 'tao_energy_measurements', 'quad', 'quad_corrector', 'solenoid'])

In [8]:
good_names = set(df['name'])
bad_eles = [] # any bad eles

for ele in bad_eles:
    good_names.remove(ele)

DM = {}
for name, dm in DM0.items():
    if name == 'tao_energy_measurements':
        # don't filter
        DM[name] = dm
    else:
        DM[name] = filter_datamap(dm, good_names)

DM['cavities']

TabularDataMap(data=   bmad_name              pvname_rbv              pvname     bmad_factor  \
0    CAVL011  ACCL:L0B:0110:PACTMEAN  ACCL:L0B:0110:PDES        0.002778   
1    CAVL012  ACCL:L0B:0120:PACTMEAN  ACCL:L0B:0120:PDES        0.002778   
2    CAVL013  ACCL:L0B:0130:PACTMEAN  ACCL:L0B:0130:PDES        0.002778   
3    CAVL014  ACCL:L0B:0140:PACTMEAN  ACCL:L0B:0140:PDES        0.002778   
4    CAVL015  ACCL:L0B:0150:PACTMEAN  ACCL:L0B:0150:PDES        0.002778   
5    CAVL016  ACCL:L0B:0160:PACTMEAN  ACCL:L0B:0160:PDES        0.002778   
6    CAVL017  ACCL:L0B:0170:PACTMEAN  ACCL:L0B:0170:PDES        0.002778   
7    CAVL018  ACCL:L0B:0180:PACTMEAN  ACCL:L0B:0180:PDES        0.002778   
8      BUN1B  ACCL:GUNB:455:PACT_AVG  ACCL:GUNB:455:PDES        0.002778   
9    CAVL011  ACCL:L0B:0110:AACTMEAN                None  1000000.000000   
10   CAVL012  ACCL:L0B:0120:AACTMEAN                None  1000000.000000   
11   CAVL013  ACCL:L0B:0130:AACTMEAN                None  1000000.00

In [9]:
# datamaps to exclude
DENYLIST = [
    #'bpms',
   # 'cavities',
    'correctors',
    #'quad',
]

In [10]:
# PVs needed
PVLIST =  []
for name, dm in DM.items():
    if name in DENYLIST:
        continue
    PVLIST.extend(dm.pvlist)
PVLIST = list(set(PVLIST))
len(PVLIST)
PVLIST[0:10]

['BPMS:HTR:760:Y',
 'BPMS:HTR:830:Y',
 'BPMS:DIAG0:285:X',
 'BPMS:COL0:135:X',
 'BPMS:DIAG0:330:X',
 'QUAD:HTR:140:BACT',
 'BPMS:GUNB:314:TMIT',
 'BPMS:HTR:980:X',
 'BPMS:DIAG0:270:TMIT',
 'BPMS:DIAG0:520:TMIT']

## EPICS

In [11]:
import epics
from epics import caget_many, caget
from time import sleep, time

def caget_dict(pvlist):
    return dict(zip(pvlist, caget_many(pvlist)))

In [12]:
# Test get
PVDATA = caget_dict(PVLIST)
BAD_DEVICES = set()
PVLIST_GOOD = []
for k, v in PVDATA.items():
    if v is None:
        print('Bad PV:', k)
        device = ':'.join((k.split(':')[:-1]))
        BAD_DEVICES.add(device)
    else:
        PVLIST_GOOD.append(k)
        
# Get bmad names
bdf = devices[['alias', 'name']].set_index('alias')
BAD_NAMES = list(bdf.loc[list(BAD_DEVICES)]['name'])
BAD_NAMES        
        
BAD_DEVICES, BAD_NAMES   

Bad PV: BPMS:DIAG0:330:X
Bad PV: BPMS:HTR:980:X
Bad PV: BPMS:DIAG0:330:Y
Bad PV: BPMS:HTR:980:Y
Bad PV: BPMS:HTR:980:TMIT
Bad PV: BPMS:DIAG0:330:TMIT


({'BPMS:DIAG0:330', 'BPMS:HTR:980'}, ['BPMHD04', 'BPMDG0RF'])

In [13]:
MONITOR = {pvname:epics.PV(pvname) for pvname in PVLIST_GOOD}
sleep(1) # Wait for all to connect

In [14]:
def get_pvdata():   
    itime = isotime()
    pvdata =  {k:MONITOR[k].get() for k in MONITOR}
    return pvdata
PVDATA = get_pvdata()

# Fiter datamamps again

In [15]:
for name in BAD_NAMES:
    good_names.remove(name)

In [16]:
DM_GOOD = {}
for name, dm in DM.items():
    if name == 'tao_energy_measurements':
        # don't filter
        DM_GOOD[name] = dm
    else:
        DM_GOOD[name] = filter_datamap(dm, good_names)

# Tao conveniences

In [17]:
def tao_commands(pvdata):
    cmds = []
    for name, dm in DM_GOOD.items():
        cmds.extend(dm.as_tao(pvdata))
    return cmds

In [18]:
def save_cmds(cmds, filename='cmds.tao'): # Write to file for running with vanilla Tao
    with open(filename, 'w') as f:
        f.write('set global lattice_calc_on = F\n')
        f.write('set global plot_on = F\n')    
        for cmd in CMDS:
            f.write(cmd+'\n')
        f.write('set global lattice_calc_on = T\n')        
        f.write('set global plot_on = T\n')   

In [19]:
# Match HTR to design
def set_htr_twiss(tao):
    cmds="""
vv
vd
use dat HTR.begtwiss[1:4]
use var begtwiss[1:4]
olmdif
""".split('\n') 
    tao.cmds(cmds)
    tao.cmd('set global lattice_calc_on = T')
    tao.cmd('run')
    tao.cmd('set global plot_on = T')
#set_htr_twiss(tao)

In [20]:
# mat2 x, y for PyEmittance
def get_mats():
    mat6 = tao.matrix('Q0H01#2', 'OTR0H04')['mat6']
    mat2x = mat6[0:2, 0:2]
    mat2y = mat6[2:4, 2:4]
    return mat2x, mat2y

## Form commands using PVDATA and datamaps

In [21]:
# Master switches for element scaling
tao.cmd('set ele quad::* field_master = T')

# Turn off phase overlays 
tao.cmd('set ele SC_L* is_on = F', raises=False)

if MODEL == 'sc_inj':
    tao.cmd('set ele lcavity::* autoscale_phase = T')

# Continuous loop

In [22]:
def run1():
    #sleep(.001)
    t1 = time()
    pvdata = get_pvdata()
    cmds = tao_commands(pvdata)
    tao.cmd('set global plot_on = F;set global lattice_calc_on = F')
    tao.cmds(cmds); # Apply

    tao.cmd('set global lattice_calc_on = T')
    tao.cmd('set global plot_on = T')
    #toggle_beam()
    
    dt = time()-t1
    #print(dt)    


In [23]:
%%time
run1()

CPU times: user 23.9 ms, sys: 2.26 ms, total: 26.2 ms
Wall time: 29.8 ms


In [24]:
# Set twiss
set_htr_twiss(tao)

In [25]:
# Check charge (pC)
df = pd.DataFrame(tao.data_d_array('orbit', 'charge'))
df['charge_live'] =df['meas_value']*1e12
df[['ele_name', 'charge_live']]

,ele_name,charge_live
0,BPM1B,78.840369
1,BPM2B,65.290631
2,BPM0H01,63.795613
3,BPM0H04,55.929705
4,BPM0H05,65.769464
5,BPM0H08,63.446005
6,BPMH1,63.412162
7,BPMH2,23.395678
8,BPMHD01,5.000936
9,BPMHD02,2.479826


In [26]:
%%tao
use dat orbit.x
use dat orbit.y
sc floor -5 5
sc top 0 80


-------------------------
Tao> use dat orbit.x
  orbit.x[1:23]                                  Using: 1:11 13:18 20:23
  orbit.y[1:23]                                  Using:
  orbit.charge[1:23]                             Using:
-------------------------
Tao> use dat orbit.y
  orbit.x[1:23]                                  Using: 1:11 13:18 20:23
  orbit.y[1:23]                                  Using: 1:11 13:18 20:23
  orbit.charge[1:23]                             Using:
-------------------------
Tao> sc floor -5 5
-------------------------
Tao> sc top 0 80
-------------------------
Tao> 


# Continuous run
    

In [ ]:
# Run forever
while True:
    run1()